In [89]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import geopandas
import random
import matplotlib.pyplot as plt
import math

from mip import *
import timeit

from stable_baselines3 import DQN, TD3, DDPG

## Define Presets and Result File

In [90]:
alloc_type = 'server' # 'server':to fix number of servers and vary users, 'user': to fix number of users and vary servers
latency_threshold = 10
S = 2 #number of services


#Result file details
if alloc_type == 'user':
    U = 500 
    result_file = f'allocation_results/allocation_for_user{U}_thres{latency_threshold}.csv'

if alloc_type == 'server':
    N = 50
    result_file = f'allocation_results/allocation_for_server{N}_thres{latency_threshold}.csv'

column_names = ["user", "server", 
                "ilp_user", "ilp_time",
                "greedy_user", "greedy_time",
                "rl_user", "rl_time",
               ]
result_user = pd.DataFrame(columns = column_names)
result_user.to_csv(result_file, index=False)
result_user = pd.read_csv(result_file)

In [91]:
# Dataset and models
filename_base = 'dataset/dual_s_base.csv' #filename of data with only 1 user
model_rl = DQN.load(f"trained_agents/edge_agent_thres{latency_threshold}")
# model_exp = DQN.load("edge_agent_under_train")
model_td3 = TD3.load(f"trained_agents/td3_thres{latency_threshold}")
users_high = 500
users_low = 100 #to calculate the users numbers from action
users_res = 100

AttributeError: 'DQNPolicy' object has no attribute 'actor'

### Utility Function to Load EUA Data

In [67]:
#Add data to users
def add_data(series): #not used now
    random_speed = random.uniform(0,1)
    if random_speed <= 0.3:
        series['Speed']= 0
    if random_speed > 0.3 and random_speed <= 0.6:   
        series['Speed']= random.uniform(1, 2)
    if random_speed > 0.6:
        series['Speed'] = random.uniform(10, 20)
    series['Direction']= random.randrange(0, 360, 10)
    qospref = [ random.randint(1,3), random.randint(1,3) ]
    qospref.sort()
    series['LowQoS'] = qospref[0]
    series['HighQoS'] = qospref[1]
    return series

#================Load Planet Lab data
def load_planetlab():
    #Convert to triangle
    ldata = np.loadtxt('eua/PlanetLabData_1')[np.tril_indices(490)]
    ldata = ldata[ ldata != 0]
    ldata =np.unique(ldata)
    np.set_printoptions(suppress=True)
    length = ldata.shape[0]
    latency_row = 150
    latency_col = (length//latency_row) #Global Data used
    ldata = np.resize(ldata, latency_col*latency_row)
    latency = ldata.reshape(latency_row,-1)
    return latency

#=================Fetch Network latency
def fetch_network_lat(distance, latency_data):
    rep_lat = np.random.choice(latency_data[distance], size=1, replace=True)
    return rep_lat/1000 #convert latency to seconds
    

#===============User Data
def load_users(num_of_users):
    user_raw = pd.read_csv("eua/users.csv")
    user_raw = user_raw.rename_axis("UID")
    df = user_raw.sample(num_of_users)
#     user_raw = user_raw.reset_index()
    gdf = geopandas.GeoDataFrame(df, geometry = geopandas.points_from_xy(df.Longitude, df.Latitude), crs = 'epsg:4326')
    user = gdf [['geometry']] #Keep Geometry column
    user = user.to_crs(epsg=28355) #Covert the format of data
    #Insert additional data to dataframe
    #user = user.apply(add_data, axis=1)
    return user

#================Server Data
def load_servers(num_of_servers):
    server_raw = pd.read_csv("eua/servers.csv")
    server_raw = server_raw.rename_axis("SID")
    df = server_raw.sample(num_of_servers) #Sample Servers Randomly
    gdf = geopandas.GeoDataFrame(df, geometry = geopandas.points_from_xy(df.LONGITUDE, df.LATITUDE), crs = 'epsg:4326')
    server = gdf [['geometry']] #Keep Geometry column
    server = server.to_crs(epsg=28355) #Cover to crs in Australian EPSG
    def add_radius(series):
#         radius = random.randrange(150, 250, 10)
        radius = 150 #radus fixed to 150
        series.geometry = series.geometry.buffer(radius)
        #series['radius'] = radius    '''FIXED'''
#         series['resource'] = tcomp
        return series
    server = server.apply(add_radius, axis = 1)
    return server

#================neighbourhood Computing
def ngb_matrix(U, N, S): 
    #U: number of users
    #N: number of servers
    #S: number of services
    # U X N matrix
    user = load_users(U)
    server = load_servers(N)
    neighbourhood = np.zeros([U, N]) #matrix user(row) vs server(col)
    network_latency = np.zeros(N) #network latency for each server
    latency_data = load_planetlab() #load planetlad data, return matrix of bin size 150
    
    for u in range(0, U):
        for n in range(0, N):
            if server.iloc[n].geometry.contains(user.iloc[u].geometry):
                neighbourhood[u,n]=1
                #compute distance and assign latency
                distance = server.iloc[n].geometry.centroid.distance(user.iloc[u].geometry)
                rep_lat = fetch_network_lat(int(distance), latency_data) #fetch latency according to distance
                if network_latency[n] < rep_lat:
                    network_latency[n] = rep_lat
                
            else:
                neighbourhood[u,n]=0
    
    service = np.zeros(U)
    
    #assign service to the users
    for u in range(0, U):
        service[u] = random.randrange(0, S, 1) #generate a random service request to the user
        
    #find service request in each server
    server_service = np.zeros((N, S))
    
    #find the number of service request in each server
    for n in range(0, N):
        for u in range(0, U):
            if neighbourhood[u][n] == 1:
                server_service[n][int(service[u])] += 1
                
                
    
    return neighbourhood, user, server, service, server_service, network_latency

#Plot the user and server dataset on the map
def plot_data(user, server):
    %config InlineBackend.figure_format='retina'
    %matplotlib inline

    cbd = geopandas.read_file('eua/maps', crs = {'init': 'epsg=28355'} ) #read cbd-australia location data

    fig, ax = plt.subplots(1, 1, figsize=(15,10))
    ax.set_aspect('equal')

    ax.set_xlim(319400, 322100)
    ax.set_ylim(5811900, 5813700)

    user.plot(ax=ax, marker='o', color='red', markersize=20, zorder=3, label="users")
    server.plot(ax =ax, linestyle='dashed', edgecolor='green', linewidth=1, facecolor="none", zorder=1)
    server.centroid.plot(ax=ax, marker='s', color='blue', markersize=50, zorder=2, label="server")
    cbd.plot(ax=ax, color='grey', zorder=0, alpha = 0.3);

    ax.set_title("MEC Environment(EUA): CBD Melbourne(Australia)")
    ax.legend(bbox_to_anchor=(1, 0), loc='lower left')
    plt.show()

## Load data for single user execution

In [68]:
def generate_server_state(num_server):
    df = pd.read_csv(filename_base)
    
#     df['ram'] = df['ram'].div(1000).round(0).astype(int)
#     df['workload_cpu'] = df['workload_cpu'].div(10).round(0).astype(int)
    df['workload_gpu'] = df['workload_gpu'].multiply(1/80).round(0).astype(int) #round gpu workload
#     df['users_yolo'] = df['users_yolo'].div(100).round(0).astype(int)
#     df['users_mnet'] = df['users_mnet'].div(100).round(0).astype(int)
    
    
    #get unique data in set
    ram = df.ram.unique()
    cores = df.cores.unique()
    workload_cpu = df.workload_cpu.unique()

    
    server_state = []
    gamma = []
    
    for s_id in range(num_server):
        #generate unique state
        gram = np.random.choice(ram, 1)[0]
        gcores = np.random.choice(cores, 1)[0]
        gwl_c = np.random.choice(workload_cpu, 1)[0]
    
        
       
        #fetch gamma for the state
        fetch_state = df.loc[ (df['ram'] == gram) & (df['cores']== gcores) & (df['workload_cpu']==gwl_c) ]
       
        gwl_g = fetch_state.sample().iloc[0]['workload_gpu'] #fetch workload randmoly
        fetch_time = fetch_state.loc[ (df['workload_gpu'] == gwl_g) ]
        
        time_yolo = fetch_time['time_yolo'].mean() #average of time for particular state
        time_mnet = fetch_time['time_mnet'].mean()
        
        gs1 = server_service[s_id][0] #assignt the state according to service request to each server
        gs2 = server_service[s_id][1] 
        server_state.append( [gram, gcores, gwl_c, gwl_g, gs1, gs2] )
        
        gamma.append((time_yolo, time_mnet)) #append the gamma value of each server
    
    return server_state, gamma

# Algorithms

### ILP Algorithm

In [69]:
def ilp_algo():
    ## ===================================ILP with python mip
    # >> solver_name=GRB
    # >> Currently using CBC
    I = range(U) #user
    J = range(N) #server

    alloc = Model(sense=MAXIMIZE, name="alloc", solver_name=CBC)

    def coverage(user_ix, server_ix):
        if ngb[user_ix][server_ix]==1:
            return 1
        else:
            return 0

    #U: num of users, N: num of servers
    x = [[ alloc.add_var(f"x{i}{j}", var_type=BINARY) for j in J] for i in I]

    #Objective Equation
    alloc.objective = xsum( x[i][j]  for i in I for j in J )

    #1 .Coverage Constraint
    for i in I:
        for j in J:        
                if not coverage(i,j):
                    alloc += x[i][j] == 0

    #2. User allocation to single server constrain
    for i in I:
        alloc += xsum( x[i][j] for j in J ) <=1

    #3. Latency Constraint
    for j in J:
        alloc += xsum( gamma[j][int(service[i])]*x[i][j] for i in I ) <=latency_threshold-network_latency[j] 

    #alloc.write("test-model.lp")

    #===========Start Optimization=========
    alloc.optimize(max_seconds=25)

    #==========ILP Ends here
    #print(f"Number of Solutions:{qoe.num_solutions}")
    ilp_allocation = [ (i,j) for i in I for j in J if x[i][j].x >= 0.99]

    #print(f"Number of Solutions:{qoe.num_solutions}")
    #print(f"Objective Value:{qoe.objective_value}")
    allocated_num_users = len(ilp_allocation)
    print("ILP Allocated Num of Users: {}".format(allocated_num_users))
    # selected.sort()
    return ilp_allocation

### RL Allocation Algorithm

In [70]:
#Load model
def rl_algo():
    #for each server find the number of possible users using model
    server_capacity = np.zeros((N, S))
    for server_id in range(N):
        state = server_state[server_id]
#         if model_type == 'lin':
        action = model_rl.predict(np.array(state), deterministic=True)
#         if model_type == 'exp':
#             action = model_exp.predict(np.array(state), deterministic=True)
        u1 = action[0]//5 + 1
        u2 = (action[0]+1) - (u1-1)*5
        server_capacity[server_id][0] = u1*100 #model output
        server_capacity[server_id][1] = u2*100 #model output
    
    #sort with number of servers per user
    col1 = np.array([np.sum(ngb,axis=1)])
    col2 = np.array([np.arange(U)])
    sorted_ngb = np.concatenate((ngb, col1.T, col2.T), axis=1) #add rowsum and index column
    sorted_ngb = sorted_ngb[np.argsort(sorted_ngb[:, N])] #sort the rows based on rowsum column

    #allocation heuristic
    #run allocation algorithm
    rl_allocation = []

    for i in range(U):
        server_list = np.where(sorted_ngb[i, :N] == 1)[0] #get the list of server to which user is connected

        if len(server_list) == 0: #skip users with no server
            continue
        
        ser = int(service[i]) #which service user is requesting
        choosen_server = server_list[np.argmax(server_capacity[server_list, ser])] #find the id of choosen server
        
        
        if  server_capacity[choosen_server][ser] > 0: #assigning user to choosen_server
            server_capacity[choosen_server][ser] -= 1 #decrement the server capacity

            rl_allocation.append( (int(sorted_ngb[i, N+1]), choosen_server) ) #(user, server) alloc pair
    print('RL Num of allocation: {}'.format(len(rl_allocation)))

    return rl_allocation

In [ ]:
#Load model
def td3_algo():
    #for each server find the number of possible users using model
    server_capacity = np.zeros((N, S))
    for server_id in range(N):
        state = server_state[server_id]
#         if model_type == 'lin':
        action = model_td3.predict(np.array(state), deterministic=True)
#         if model_type == 'exp':
#             action = model_exp.predict(np.array(state), deterministic=True)
        u1 = action[0]//5 + 1
        u2 = (action[0]+1) - (u1-1)*5
        server_capacity[server_id][0] = u1*100 #model output
        server_capacity[server_id][1] = u2*100 #model output
    
    #sort with number of servers per user
    col1 = np.array([np.sum(ngb,axis=1)])
    col2 = np.array([np.arange(U)])
    sorted_ngb = np.concatenate((ngb, col1.T, col2.T), axis=1) #add rowsum and index column
    sorted_ngb = sorted_ngb[np.argsort(sorted_ngb[:, N])] #sort the rows based on rowsum column

    #allocation heuristic
    #run allocation algorithm
    rl_allocation = []

    for i in range(U):
        server_list = np.where(sorted_ngb[i, :N] == 1)[0] #get the list of server to which user is connected

        if len(server_list) == 0: #skip users with no server
            continue
        
        ser = int(service[i]) #which service user is requesting
        choosen_server = server_list[np.argmax(server_capacity[server_list, ser])] #find the id of choosen server
        
        
        if  server_capacity[choosen_server][ser] > 0: #assigning user to choosen_server
            server_capacity[choosen_server][ser] -= 1 #decrement the server capacity

            rl_allocation.append( (int(sorted_ngb[i, N+1]), choosen_server) ) #(user, server) alloc pair
    print('RL Num of allocation: {}'.format(len(rl_allocation)))

    return rl_allocation

### Nearest Neighbourhood Greedy Algorithm

In [71]:
def greedy_algo():
    server_capacity = np.zeros(N)
    rl_allocation = []
    for user_id in range(U):
        server_ngb_list = np.where(ngb[user_id, :N] == 1)[0] #get the list of server to which user is connected
        if len(server_ngb_list) == 0: #ignore the users which are not under any servers
            continue 
    
        #find the distance to each users in the server_ngb_list
        dist_list = np.array([ server_ngb_list, [server.iloc[i]['geometry'].centroid.distance(user.iloc[user_id]['geometry']) for i in server_ngb_list] ])
        # sorted list of servers based on the distance from users
        sorted_distance_list = dist_list[ :, dist_list[1].argsort()]
        #get the list of servers arranged in least to max distance
        server_list = sorted_distance_list[0].astype(int)
        
        lat = 0
        for server_id in server_list:
            lat = gamma[server_id][int(service[user_id])]
            if server_capacity[server_id]+lat <= latency_threshold-network_latency[server_id]:
                server_capacity[server_id] += lat #increment the server_capacity of server
                rl_allocation.append( (user_id, server_id) ) #(user, server) alloc pair
                break
    
    print('Greedy-Ngb Num of allocation: {}'.format(len(rl_allocation)))
    return rl_allocation

# main(): Iterative Loop to generate results for various user server configuration

In [72]:
if alloc_type == 'user':
    for N in range(20, 100, 20):
    #     for N in range(20, 100, 20):
        for epoch in range(50):
            print("User:", U, 'Server:', N, 'Epoch:', epoch)
    #             U = 50 #Number of users #Define U, N and ngb Matrix
    #             N = 10 #Number of Servers
            ngb, user, server, service, server_service, network_latency = ngb_matrix(U, N, S) #generate server and user from EUA data #determine neighbourhood matrix
            server_state, gamma = generate_server_state(N) #assign state and gamma for each user

            #=======ILP starts
            start = 0
            stop = 0
            execution_time_ilp = 0
            start = timeit.default_timer()

            ilp_aloc = ilp_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_ilp = stop - start
            #========ILP ends

            #=======Greedy starts
            start = 0
            stop = 0
            execution_time_greedy = 0
            start = timeit.default_timer()

            greedy_aloc = greedy_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_greedy = stop - start
            #========Greedy ends

            #=======RL starts
            start = 0
            stop = 0
            execution_time_rl = 0
            start = timeit.default_timer()

            rl_aloc = rl_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_rl = stop - start
            #========RL_linear ends

            #=======td3 starts
            start = 0
            stop = 0
            execution_time_rl = 0
            start = timeit.default_timer()

            td3_aloc = td3_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_td3 = stop - start
            #========RL_linear ends

            #========Store results to file
            to_append = [U, N,
                         len(ilp_aloc), execution_time_ilp,
                         len(greedy_aloc), execution_time_greedy,
                         len(rl_aloc), execution_time_rl,
                         len(td3_aloc), execution_time_td3,
                        ] 
            dseries = pd.Series(to_append, index = result_user.columns)
            result_user = result_user.append(dseries, ignore_index=True)
            print("epoch:", epoch)
    result_user.to_csv(result_file, index=False)

In [73]:
result_user

,user,server,ilp_user,ilp_time,greedy_user,greedy_time,rl_user,rl_time


In [82]:
if alloc_type == 'server': #server fix vary number of users
    for U in range(100, 600, 100):
        for epoch in range(1):
            print("\n---------------------------------------")
            print("User:", U, 'Server:', N, 'Epoch:', epoch)
    #             U = 50 #Number of users #Define U, N and ngb Matrix
    #             N = 10 #Number of Servers
            ngb, user, server, service, server_service, network_latency = ngb_matrix(U, N, S) #generate server and user from EUA data #determine neighbourhood matrix
            server_state, gamma = generate_server_state(N) #assign state and gamma for each user

            #=======ILP starts
            start = 0
            stop = 0
            execution_time_ilp = 0
            start = timeit.default_timer()

            ilp_aloc = ilp_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_ilp = stop - start
            #========ILP ends

            #=======Greedy starts
            start = 0
            stop = 0
            execution_time_greedy = 0
            start = timeit.default_timer()

            greedy_aloc = greedy_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_greedy = stop - start
            #========Greedy ends

            #=======RL_linear starts
            start = 0
            stop = 0
            execution_time_rl = 0
            start = timeit.default_timer()

            rl_aloc = rl_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_rl = stop - start
            #========RL_linear ends


           #=======td3 starts
            start = 0
            stop = 0
            execution_time_rl = 0
            start = timeit.default_timer()

            td3_aloc = td3_algo() #call ILP algorithm

            stop = timeit.default_timer()
            execution_time_td3 = stop - start
            #========RL_linear ends

            #========Store results to file
            to_append = [U, N,
                         len(ilp_aloc), execution_time_ilp,
                         len(greedy_aloc), execution_time_greedy,
                         len(rl_aloc), execution_time_rl,
                         len(td3_aloc), execution_time_td3,
                        ] 
            dseries = pd.Series(to_append, index = result_user.columns)
            result_user = result_user.append(dseries, ignore_index=True)
            print("epoch:", epoch)
    result_user.to_csv(result_file, index=False)


---------------------------------------
User: 100 Server: 50 Epoch: 0
ILP Allocated Num of Users: 81
Greedy-Ngb Num of allocation: 81
RL Num of allocation: 81
epoch: 0

---------------------------------------
User: 200 Server: 50 Epoch: 0
ILP Allocated Num of Users: 150
Greedy-Ngb Num of allocation: 150
RL Num of allocation: 150
epoch: 0

---------------------------------------
User: 300 Server: 50 Epoch: 0
ILP Allocated Num of Users: 258
Greedy-Ngb Num of allocation: 253
RL Num of allocation: 260
epoch: 0

---------------------------------------
User: 400 Server: 50 Epoch: 0
ILP Allocated Num of Users: 314
Greedy-Ngb Num of allocation: 297
RL Num of allocation: 328
epoch: 0

---------------------------------------
User: 500 Server: 50 Epoch: 0
ILP Allocated Num of Users: 415
Greedy-Ngb Num of allocation: 393
RL Num of allocation: 427
epoch: 0


In [84]:
result_user

,user,server,ilp_user,ilp_time,greedy_user,greedy_time,rl_user,rl_time
0,100.0,50.0,80.0,0.131357,80.0,0.015563,80.0,0.007920
1,100.0,50.0,89.0,0.120040,89.0,0.016793,89.0,0.007678
2,100.0,50.0,87.0,0.123697,87.0,0.016144,87.0,0.007591
3,100.0,50.0,72.0,0.137681,72.0,0.013875,72.0,0.007632
4,100.0,50.0,80.0,0.122100,80.0,0.015711,80.0,0.007986
...,...,...,...,...,...,...,...,...
221,100.0,50.0,81.0,0.105348,81.0,0.015634,81.0,0.007380
222,200.0,50.0,150.0,0.395905,150.0,0.030647,150.0,0.007697
223,300.0,50.0,258.0,0.354603,253.0,0.045962,260.0,0.008416
224,400.0,50.0,314.0,0.600479,297.0,0.055001,328.0,0.008886


end